Multi Vector Retriever

Smaller Chunk

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

In [2]:
with open('../langchain/lcel/custom_embedding/data/Toronto.txt', 'r') as f:
    html_toronto = f.read()
with open('../langchain/lcel/custom_embedding/data/San_Francisco.txt', 'r') as f:
    html_sf = f.read()

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
toronto_header_splits = html_splitter.split_text(html_toronto)
sf_header_splits = html_splitter.split_text(html_sf)

In [3]:
toronto_header_splits[0].metadata = {"Header2": "Overview", "Header 3": "population"}
sf_header_splits[0].metadata = {"Header2": "Overview", "Header 3": "population"}
total_headers = toronto_header_splits + sf_header_splits

In [4]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(total_headers)

In [5]:
len(docs)

75

In [6]:
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
client= QdrantClient()
embeddings = OpenAIEmbeddings()
collection = "toronto_sf_chunk_400"


In [29]:
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [30]:
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
id_key = "doc_id"
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [31]:
len(sub_docs)

716

In [36]:
qdrant= Qdrant.from_documents(sub_docs, path="./toronto_sf_chunk400",client=client, collection_name=collection, embedding=embeddings)


In [37]:
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever

#parent doc chunk 1,000
store = InMemoryStore()
retriever = MultiVectorRetriever(
    vectorstore=qdrant,
    docstore=store,
    id_key=id_key,
)

In [38]:
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [45]:
relevant_docs = retriever.vectorstore.similarity_search("population of San Francisco")

In [46]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
len(relevant_docs)

4

In [47]:
pretty_print_docs(relevant_docs)

Document 1:

The 2020 United States census showed San Francisco's population to be 873,965, an increase of 8.5% from the 2010 census. With roughly one-quarter the population density of Manhattan, San Francisco is the second-most densely populated large American city, behind only New York City among cities greater than 200,000 population, and the fifth-most densely populated U.S. county, following only four of
----------------------------------------------------------------------------------------------------
Document 2:

San Francisco is part of the five-county San Francisco–Oakland–Hayward, CA Metropolitan Statistical Area, a region of 4.7 million people (13th most populous in the U.S.), and has served as its traditional demographic focal point. It is also part of the greater 14-county San Jose-San Francisco-Oakland, CA Combined Statistical Area, whose population is over 9.6 million, making it the fifth-largest
--------------------------------------------------------------------------

We can see that with the same "comparing population" the retrieving result is very bad, it completely ignores San Francisco. The advantage of is the document is much smaller comparing to a bigger chunk

Summary
We will summary each chunk and embed + store it along with the Document.